In [ ]:
#@title Installing dependencies:
# installing foundry:
!curl -L https://foundry.paradigm.xyz | bash
!source ~/.bashrc
import os
os.environ['PATH'] += ":/root/.foundry/bin"
!foundryup

# cloning repo and installing foundry dependencies:
!git clone -b main https://github.com/baking-bad/etherlink-bridge.git
%cd etherlink-bridge/
!git submodule update --init --recursive

# installing python dependencies:
!pip install poetry
!poetry install

### Environment Initialization:

Run the following cell to set up the environment and provide private keys for interacting *with* the bridge. Default values are provided for all variables; however, you may wish to deploy your own token and establish a bridge using your own keys.

In [2]:
!poetry run init_wallets

Enter "L1_PRIVATE_KEY" (default: edsk2nu78mRwg4V5Ka7XCJFVbVPPwhry8YPeEHRwzGQHEpGAffDvrH): 
Enter "L1_PUBLIC_KEY_HASH" (default: tz1YG6P2GTQKFpd9jeuESam2vg6aA9HHRkKo): 
Enter "L1_RPC_URL" (default: https://rpc.tzkt.io/nairobinet/): 
Enter "L1_ROLLUP_ADDRESS" (default: sr19t5DDEhvnPaNZgNYVAkmBRd5nAPyMU7nP): 
Enter "L2_PRIVATE_KEY" (default: 8636c473b431be57109d4153735315a5cdf36b3841eb2cfa80b75b3dcd2d941a): 
Enter "L2_PUBLIC_KEY" (default: 0xBefD2C6fFC36249ebEbd21d6DF6376ecF3BAc448): 
Enter "L2_MASTER_KEY" (default: 9722f6cc9ff938e63f8ccb74c3daa6b45837e5c5e3835ac08c44c50ab5f39dc0): 
Enter "L2_RPC_URL" (default: https://etherlink.dipdup.net): 
Enter "L2_ROLLUP_RPC_URL" (default: https://etherlink-rollup-nairobi.dipdup.net): 
Enter "L2_KERNEL_ADDRESS" (default: 0x0000000000000000000000000000000000000000): 
Enter "L2_WITHDRAW_PRECOMPILE_ADDRESS" (default: 0x0000000000000000000000000000000000000040): 


### Bridge token pair listing:

The **Ticket Transport Layer**, a key component of the bridge, enables ticket transfers between Tezos and Etherlink. To list a new token pair and establish a bridge, users need to:
1. If the token is an **FA2** or **FA1.2** standard token that doesn’t natively support tickets, the user must deploy a **Ticketer** contract on the Tezos side. This contract links to the specific Tezos token to convert it into a ticket.
2. The user then deploys an **ERC20Proxy** on the Etherlink side – a smart contract implementing ERC20. This contract should be configured to expect tickets from the deployed **Ticketer** including ticketer address and ticket content. This setup allows the rollup to mint ERC20 tokens corresponding to the incoming tickets from Tezos.
3. Additionally, deploy a **TicketHelper** on the Tezos side, targeting the specific token and **Ticketer** pair. This step is necessary as the **transfer_ticket** operation is not currently supported by wallets. The **TicketHelper** facilitates the wrapping of tokens into tickets and their transfer to the rollup in a single transaction.

#### Deploying a Token

For demonstration purposes, users can deploy a test token that will later be bridged. The bridge has been tested with two types of tokens, which are available in the repository:
- The **FA1.2** **Ctez** token.
- The **FA2** **fxhash** token.

To deploy this token and allocate the total supply to the token's originator, execute the following script. The example below illustrates the deployment for an **FA2** token type:

In [ ]:
!poetry run deploy_token --token-type FA2

deploying contract from filename /content/etherlink-bridge/etherlink-bridge/tezos/tests/helpers/../tokens/fa2-fxhash.tz
found contract address: KT1EMyCtaNPypSbz3qxuXmNZVfvhqifrf5MR


#### Deploying a Ticketer

To deploy a ticketer for a specific token address and token id, use the following script. The example below uses a **FA1.2** token previously deployed on Nairobinet:

In [ ]:
!poetry run deploy_ticketer --token-address KT1EMyCtaNPypSbz3qxuXmNZVfvhqifrf5MR --token-type FA2 --token-id 0

deploying contract from filename /content/etherlink-bridge/etherlink-bridge/tezos/tests/helpers/../../build/ticketer.tz
found contract address: KT1MauRYJiXxD7a8iZkhpdnc4jHu7iGGXDbs
address_bytes: 018ea031e382d5be16a357753fb833e609c7d2dd9b00
content_bytes: 0707000005090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a00000016013f65105866518de12034c340e2b2f65d80780c580007040100000008746f6b656e5f69640a000000030500000704010000000a746f6b656e5f747970650a00000009050100000003464132


During the Ticketer's deployment, user will receive its parameters, including **address_bytes** and **content_bytes**. These are required for the origination of the **ERC20Proxy**.

#### Deploying a Ticket Helper

To enable Tezos wallets to interact with specific tickets, deploy a Ticket Helper using the script below. The example provided uses a ticketer that was previously deployed in Nairobinet:

In [ ]:
!poetry run deploy_ticket_helper --ticketer-address KT1MauRYJiXxD7a8iZkhpdnc4jHu7iGGXDbs

deploying contract from filename /content/etherlink-bridge/etherlink-bridge/tezos/tests/helpers/../../build/ticket-helper.tz
found contract address: KT18nod2GU8PzYcrkspXz7dFkGisThZpgdLW


#### Deploying ERC20Proxy

Finally, to deploy a token contract on the Etherlink side, which will mint tokens upon deposit, execute the script below. This script requires **ticketer-address-bytes** and **ticketer-content-bytes**. The example uses bytes from a ticketer previously deployed in Nairobinet:

In [ ]:
!poetry run deploy_erc20 --ticketer-address-bytes 018ea031e382d5be16a357753fb833e609c7d2dd9b00 --ticket-content-bytes 0707000005090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a00000016013f65105866518de12034c340e2b2f65d80780c580007040100000008746f6b656e5f69640a000000030500000704010000000a746f6b656e5f747970650a00000009050100000003464132 --token-name "FA2 Test Token" --token-symbol "FA2" --decimals 0

Error: 
Contract was not deployed



NOTE: To obtain the **ticketer-address-bytes** and **content-bytes** for an already deployed ticketer, user can use the script below:

In [ ]:
!poetry run get_ticketer_params --ticketer KT1MauRYJiXxD7a8iZkhpdnc4jHu7iGGXDbs

address_bytes: 018ea031e382d5be16a357753fb833e609c7d2dd9b00
content_bytes: 0707000005090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a00000016013f65105866518de12034c340e2b2f65d80780c580007040100000008746f6b656e5f69640a000000030500000704010000000a746f6b656e5f747970650a00000009050100000003464132


### Deposit

To make a deposit, user need to transfer Tickets to the rollup address with attached Routing Info in the [specified format](https://gitlab.com/baking-bad/tzip/-/blob/wip/029-etherlink-token-bridge/drafts/current/draft-etherlink-token-bridge/etherlink-token-bridge.md#deposit): `| receiver | proxy |` 40 bytes payload, both receiver and proxy are standard Ethereum addresses in raw form (H160). A script is available to facilitate this process. It requires the Ticket Helper address as **ticket-helper-address**, the Etherlink ERC20Proxy contract as **proxy-address**, and the bridged amount as the **amount** variable. Here's an example:

In [ ]:
!poetry run deposit --ticket-helper-address KT18nod2GU8PzYcrkspXz7dFkGisThZpgdLW --proxy-address 0x8554cD57C0C3E5Ab9d1782c9063279fA9bFA4680 --amount 777

Succeed, transaction hash: opRNngtr5dwYG6nJuXRAWXLPmE8Y17RgadwBQEtZfu3tfT9sxVv


NOTE: This script performs two operations in bulk. The first operation approves the token for the **Ticket Helper**, and the second operation makes a call to the **Ticket Helper** **deposit** entrypoint.

### Withdrawal Process

The withdrawal process involves two key steps:
1. Initiating the withdrawal on the Etherlink side, which results in the creation of an outbox message in the rollup commitment.
2. Completing the withdrawal by executing the outbox message on the Tezos side once the commitment has settled (TODO: Clarify terminology if necessary).

#### Etherlink Withdrawal

To initiate a withdrawal, user need to call the withdrawal precompile on the Etherlink side. This requires providing the **ERC20Proxy** address along with Routing Info in a [specific format](https://gitlab.com/baking-bad/tzip/-/blob/wip/029-etherlink-token-bridge/drafts/current/draft-etherlink-token-bridge/etherlink-token-bridge.md#withdrawal), two forged contracts concatenated: `| receiver | proxy |` 44 bytes. Forged contact consists of binary suffix/prefix and body (blake2b hash digest). A script is available to facilitate the withdrawal process. It requires the ERC20 contract (which will burn tokens) as **proxy-address**, the Tezos router address (which will receive the ticket from the rollup) as **router-address**, and the bridged amount as **amount**. Additionally, the **ticketer-address-bytes** and **ticketer-content-bytes** are required to allow **ERC20Proxy** validate token before burning.

NOTE: For automatic unwrapping of Tezos tickets back into tokens, the **Ticketer** address can be provided as the **router-address**.

In [ ]:
!poetry run withdraw --proxy-address 0x8554cD57C0C3E5Ab9d1782c9063279fA9bFA4680 --amount 108 --ticketer-address-bytes 018ea031e382d5be16a357753fb833e609c7d2dd9b00 --ticket-content-bytes 0707000005090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a00000016013f65105866518de12034c340e2b2f65d80780c580007040100000008746f6b656e5f69640a000000030500000704010000000a746f6b656e5f747970650a00000009050100000003464132

Error: 
tx not found, might have been dropped from mempool: 0xfc7e31241a44d3b23afdb41f5e69ecf4a8e3bc0e9f914039fe51beaa52400ed9



#### Finalizing Tezos Withdrawal

To complete the withdrawal process, user need to call the outbox message on the Tezos side once it has been finalized (settled) on the L1 side. During the withdrawal process, the Etherlink side emits a **Withdrawal** event, which includes **outboxLevel** and **outboxMsgId**.

Users can obtain **outboxLevel** and **outboxMsgId** from the **Withdrawal** event, which is emitted upon the successful formation of an outbox message. Since both parameters are of type `uint256`, to retrieve **outboxMsgId**, users need to take the last 32 bytes of the event. For **outboxLevel**, take another 32 bytes preceding those and convert them into an integer. Below is an example of how this can be done:
- Here is a link to the block explorer with the withdrawal transaction: http://blockscout.dipdup.net/tx/0xfc7e31241a44d3b23afdb41f5e69ecf4a8e3bc0e9f914039fe51beaa52400ed9/logs
- Here is **Withdrawal** event data: `0x000000000000000000000000befd2c6ffc36249ebebd21d6df6376ecf3bac4480000000000000000000000008554cd57c0c3e5ab9d1782c9063279fa9bfa468000008a7390072a389159c73687165cd7910e8a39160600000000000000000000000000000000000000000000000000000000000000000000000000000000006c00000000000000000000000000000000000000000000000000000000002920a70000000000000000000000000000000000000000000000000000000000000000`
- Here is **outboxMsgId** is the last 32 bytes: `0000000000000000000000000000000000000000000000000000000000000000`, which is integer `0`
- Here is **outboxLevel** is the 32 bytes preceding those: `00000000000000000000000000000000000000000000000000000000002920a7`, which is integer `2695335`

To simplify this process, script to parse withdrawal event added:

In [3]:
!poetry run parse_withdrawal_event --tx-hash 0xfc7e31241a44d3b23afdb41f5e69ecf4a8e3bc0e9f914039fe51beaa52400ed9

outbox_level: 2695335
outbox_index: 0


Now, when the user knows the index and outbox level where withdrawal transaction was added, it is possible to get **commitment** and **proof**, which are required for `execute_outbox_message` call. To do this users need to perform call to the `global/block/head/helpers/proofs/outbox/{outbox_level}/messages` Rollup RPC Node endpoint. There is a script which performs this call:

In [ ]:
!poetry run get_proof --level 2695335 --index 0

commitment: src12xZPCATqCQ2yEwTsdjpUrZB3j5N4QEG2yE9ooKRkdebUoNnaVc
proof: 03000246e559d1e67da67d8a58ba6336b2d0d590d907575b62bcb6f74b72a173fa1c4346e559d1e67da67d8a58ba6336b2d0d590d907575b62bcb6f74b72a173fa1c430005820764757261626c65d0e7d0dd265b0311f278277c7074a59f1e0ccd0eea95480712a8d16f2af2a5fd4803746167c00800000004536f6d650003c0eddd502da06bf12e3f51c320a2ae3e57ea25f85715bb937694c028196f5d4b40820576616c7565810370766d8107627566666572738205696e707574820468656164c00100066c656e677468c00100066f75747075740004820132810a6c6173745f6c6576656cc004002922470133810f76616c69646974795f706572696f64c00400013b0082013181086f7574626f786573010c0801061ec0b2503e758887e4c8867c03d8ee9fed3affd91be8d51b5c009fdc05dcf006c5220102ff01019500c30056c0284323d54769eace77824173c277fd6b0afc488e0c8793892d4ff56e6182445b0031c0641f1aec15551f28800a5b143a5138c20c32a6a79567e76121a03a34d7518c700019000d0009000700040004c092fc55432b8d1bb592f64e30bf689ca79cd662da42f5fa158742a5beefbe3108820732363933373233820468656164c00100066c656e677468c0

Since this outbox message settled on the L1 side, user able to execute it by making `execute_outbox_message` operation. There is a script which allows to do this using provided private keys:

In [ ]:
!poetry run execute_outbox_message --commitment src12xZPCATqCQ2yEwTsdjpUrZB3j5N4QEG2yE9ooKRkdebUoNnaVc --proof 03000246e559d1e67da67d8a58ba6336b2d0d590d907575b62bcb6f74b72a173fa1c4346e559d1e67da67d8a58ba6336b2d0d590d907575b62bcb6f74b72a173fa1c430005820764757261626c65d0e7d0dd265b0311f278277c7074a59f1e0ccd0eea95480712a8d16f2af2a5fd4803746167c00800000004536f6d650003c0eddd502da06bf12e3f51c320a2ae3e57ea25f85715bb937694c028196f5d4b40820576616c7565810370766d8107627566666572738205696e707574820468656164c00100066c656e677468c00100066f75747075740004820132810a6c6173745f6c6576656cc004002922470133810f76616c69646974795f706572696f64c00400013b0082013181086f7574626f786573010c0801061ec0b2503e758887e4c8867c03d8ee9fed3affd91be8d51b5c009fdc05dcf006c5220102ff01019500c30056c0284323d54769eace77824173c277fd6b0afc488e0c8793892d4ff56e6182445b0031c0641f1aec15551f28800a5b143a5138c20c32a6a79567e76121a03a34d7518c700019000d0009000700040004c092fc55432b8d1bb592f64e30bf689ca79cd662da42f5fa158742a5beefbe3108820732363933373233820468656164c00100066c656e677468c0010007323639353333350003810468656164c001008208636f6e74656e7473810130c0e9000000e500000000e007070a0000001600008a7390072a389159c73687165cd7910e8a39160607070a00000016018ea031e382d5be16a357753fb833e609c7d2dd9b0007070707000005090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a00000016013f65105866518de12034c340e2b2f65d80780c580007040100000008746f6b656e5f69640a000000030500000704010000000a746f6b656e5f747970650a0000000905010000000346413200ac01018ea031e382d5be16a357753fb833e609c7d2dd9b00000000087769746864726177066c656e677468c00101e0c0fbeb6f8dbed945498bb241f2aa59b61fa41ce8f4183e87a0fb207bad530de545c0928d01251fe92a323735aad221c34a984287441f9928653781aae2ba3e1ed20ec01c04d9bbd31a532e2b64f2cea68d299bc3588607b6f3301b337a382751891ff4c0edb46d89cd791d0ba42b375b963b2d3519d61f7e6b9ad7acad6836ded1b00aa9c0aec97545fa52e4fd6d81eb6616fa7c32d86ab299b2842f8f47094e69f409ebc8c0f233f9b7aaf566f55bf84c6a0d0aaa4466f24838bc9fdf8feff9cb589906a07bc0031742da1992a697d7ad87911a3c08b7ece1bd4ee8f2ff458f2ddcc75ec2e2b5c02bd0fc4984cea88f93335810d338afcea380f16c6273a48ec617f755730319e20134810d6d6573736167655f6c696d6974c002a401047761736dd07a5feea7b7245822330d8f990fd24bf317c760800acf697a8e9289b16b05222b46e559d1e67da67d8a58ba6336b2d0d590d907575b62bcb6f74b72a173fa1c43002920a70000000000e007070a0000001600008a7390072a389159c73687165cd7910e8a39160607070a00000016018ea031e382d5be16a357753fb833e609c7d2dd9b0007070707000005090a0000007405020000006e07040100000010636f6e74726163745f616464726573730a0000001c050a00000016013f65105866518de12034c340e2b2f65d80780c580007040100000008746f6b656e5f69640a000000030500000704010000000a746f6b656e5f747970650a0000000905010000000346413200ac01018ea031e382d5be16a357753fb833e609c7d2dd9b00000000087769746864726177

Succeed, transaction hash: ongZVuFZmLaGNfF7zgMvufEE8APxAwoq7pbRv5GuJXdnqXgJZKh


### Other scripts:

- Fund account:

In [ ]:
!poetry run fund_etherlink_account

Successfully funded account:

blockHash               0x56eba9a1f53fefc4de287edec978669c5d27413d0f8f430482758d8baa7da4b6
blockNumber             1766
contractAddress         
cumulativeGasUsed       21000
effectiveGasPrice       1
gasUsed                 21000
logs                    []
logsBloom               0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
root                    
status                  1
transactionHash         0xae53190694f0943e5f8b2cb28a0ed82156cc1b198d54a762fa7d25116e0f0359
transactionIndex        0
type 